In [1]:
# Cargo los datos sobre los que quiero hacer predicciones
import cx_Oracle
import pandas as pd


In [19]:

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select periodo,product_id,sum(tn) tn from PC.L_VM_COMPLETA join L_PRODUCTOS_A_PREDECIR using(product_id) group by periodo,product_id order by product_id,periodo" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pablo\AppData\Local\Temp\ipykernel_9736\1234798557.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [20]:
print(df_pred_full.shape)

(16088, 3)


In [22]:
# convertir el periodo a datetime
df_pred_full['PERIODO'] = pd.to_datetime(df_pred_full['PERIODO'], format='%Y%m')

# Para cada producto si faltan meses entre 201701 y 201912, reemplazar con 0 el tn de esos meses para cada producto
def complete_months(df, start_date, end_date):
    # Crear un rango de fechas mensuales
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS')
    
    # Crear un DataFrame con todos los meses y los productos únicos
    products = df['PRODUCT_ID'].unique()
    all_combinations = pd.MultiIndex.from_product([products, all_months], names=['PRODUCT_ID', 'PERIODO'])
    complete_df = pd.DataFrame(index=all_combinations).reset_index()
    
    # Unir con el DataFrame original
    complete_df = complete_df.merge(df, on=['PRODUCT_ID', 'PERIODO'], how='left')
    
    # Rellenar los valores NaN con 0
    complete_df['TN'] = complete_df['TN'].fillna(0)
    
    return complete_df
# Completar los meses entre 201701 y 201912
df_complete = complete_months(df_pred_full, '2017-01-01', '2019-12-01')
print(df_complete.shape)



(28080, 3)


In [26]:
# ...existing code...

from statsmodels.tsa.statespace.sarimax import SARIMAX
# No mostar los warnings de statsmodels
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='statsmodels')


# Crear un DataFrame para almacenar los resultados de las predicciones que contenga el product_id y la predicción
df_pronostico = pd.DataFrame(columns=['PRODUCT_ID', 'TN_PREDICTED'])

for product_id, group in df_complete.groupby('PRODUCT_ID'):
    group.set_index('PERIODO', inplace=True)
    group = group.sort_index()
    
    # SARIMA: (p,d,q)x(P,D,Q,s)
    # Ejemplo: (1,1,1)x(1,1,1,12) para estacionalidad anual en datos mensuales
    try:
        model = SARIMAX(group['TN'], order=(1, 1, 0), seasonal_order=(1, 1, 0, 12))
        model_fit = model.fit(disp=False)
        # ...existing code...
        forecast = model_fit.forecast(steps=2)
        df_pronostico = pd.concat([df_pronostico,pd.DataFrame([{'PRODUCT_ID': product_id, 'TN_PREDICTED': forecast.iloc[1]}])], ignore_index=True)
# ...existing code...
    except Exception as e:
        print(f"Error for product {product_id}: {e}")

# ...existing code...

C:\Users\pablo\AppData\Local\Temp\ipykernel_9736\3275389862.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pronostico = pd.concat([df_pronostico,pd.DataFrame([{'PRODUCT_ID': product_id, 'TN_PREDICTED': forecast.iloc[1]}])], ignore_index=True)


In [27]:
# Hacer cero los valores de tn_predicted que sean menores a cero
df_pronostico['TN_PREDICTED'] = df_pronostico['TN_PREDICTED'].clip(lower=0) 
# Guardar el DataFrame de pronósticos en un archivo CSV
df_pronostico.to_csv('pronostico_sarimax_2.csv', index=False, encoding='utf-8-sig')   

In [28]:
print(df_pronostico.head())  # Mostrar las primeras filas del DataFrame de pronósticos


  PRODUCT_ID  TN_PREDICTED
0      20001   2460.547605
1      20002   1512.242450
2      20003   1585.858222
3      20004    950.804368
4      20005    844.389682
